<a href="https://colab.research.google.com/github/OvaisMemon/Function-Calling/blob/main/FunctionCallingEx01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -Uq google-generativeai>=0.7.2

In [3]:
from google.colab import userdata
import google.generativeai as genai

genai.configure(api_key=userdata.get("GOOGLE_API_KEY"))

In [4]:
def enable_lights():
    """Turn on the lighting system."""
    print("LIGHTBOT: Lights enabled.")


def set_light_color(color: str):
    """Set the light color. Lights must be enabled for this to work."""
    print(f"LIGHTBOT: Lights set to {color}.")


def stop_lights():
    """Stop flashing lights."""
    print("LIGHTBOT: Lights turned off.")

In [5]:
light_controls = [enable_lights, set_light_color, stop_lights]
instruction = "You are a helpful lighting system bot. You can turn lights on and off, and you can set the color. Do not perform any other tasks."

model = genai.GenerativeModel(
    "models/gemini-1.5-pro", tools=light_controls, system_instruction=instruction
)

chat = model.start_chat()

In [6]:
from google.generativeai.types import content_types
from collections.abc import Iterable


def tool_config_from_mode(mode: str, fns: Iterable[str] = ()):
    """Create a tool config with the specified function calling mode."""
    return content_types.to_tool_config(
        {"function_calling_config": {"mode": mode, "allowed_function_names": fns}}
    )

In [14]:
tool_config = tool_config_from_mode("none")

response = chat.send_message(
    "Hello light-bot, please turn on the lights?", tool_config=tool_config
)
print(response.text)

Turning the lights on.



In [17]:
tool_config = tool_config_from_mode("auto")

response = chat.send_message("Light this place up!", tool_config=tool_config)
print(response.parts[1])
chat.rewind();  # You are not actually calling the function, so remove this from the history.

function_call {
  name: "enable_lights"
  args {
  }
}



In [32]:
available_fns = ["set_light_color", "stop_lights", "enable_lights"]

tool_config = tool_config_from_mode("any", available_fns)

chat = model.start_chat(enable_automatic_function_calling=True)
response = chat.send_message("I want to see the lights yellow and then turn it off!", tool_config=tool_config)


LIGHTBOT: Lights enabled.
LIGHTBOT: Lights set to yellow.
LIGHTBOT: Lights turned off.


In [27]:
available_fns = ["enable_lights"]
tool_config = tool_config_from_mode("any", available_fns)

auto_chat = model.start_chat(enable_automatic_function_calling=True)
auto_chat.send_message("It's awful dark in here...", tool_config=tool_config)

LIGHTBOT: Lights enabled.


response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "Let there be light!\n"
              }
            ],
            "role": "model"
          },
          "finish_reason": "STOP",
          "avg_logprobs": -0.10337680578231812
        }
      ],
      "usage_metadata": {
        "prompt_token_count": 165,
        "candidates_token_count": 6,
        "total_token_count": 171
      }
    }),
)